# Preparación Data Final para Modelar

In [48]:
import numpy                 as np
import pandas                as pd
import matplotlib.pyplot     as plt
import seaborn               as sns

# Manejo de fechas
import datetime
import matplotlib.dates as mdates
from datetime import date
from dateutil.relativedelta import relativedelta
from sklearn import metrics

pd.set_option('display.max_columns', None)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPRegressor

#from sklearn.externals import joblib
from sklearn.utils import resample

from tensorflow import keras
from tensorflow.keras import layers

import warnings
warnings.filterwarnings('ignore')

## Datos

In [2]:
#header_list=["cliente_id","no_documento","tipo_documento","nombre_completo_cliente","nombre_cliente","primer_apellido_cliente","segundo_apellido_cliente","email_cliente","tlf_principal","tlf_celular","tlf_domicilio","tlf_empresa","tlf_otro","fecha_nacimiento_cliente","genero_cliente","estado_civil_cliente","direccion_domicilio","empresa","direccion_empresa","cargo","profesion","fecha_ultima_actualizacion","fuente_ultima_actualizacion","ciudad_cliente","segmento_cliente","sueldo_cliente","tipo_empleado","cargas_familiares_cliente","nombre","fullDate","Tiempo","fase","gestion","tipo_gestion","subtipo_gestion","subtipo_gestion1","gestion_gxc_id","tipo_gestion_gxc_id","subtipo_gestion_gxc_id","categoria","fuente_gxc","gestion_gxc_estatus","tipo_gestion_gxc_estatus","subtipo_gestion_gxc_estatus","gestionado","contactado","contactabilidad_efectiva","ganada","perdida","no_perfil","indeciso","agente_id","usuario_agente","primer_apellido_agente","segundo_apellido","nombres_agente","cedula_agente","fecha_de_nacimiento_agente","edad_agente","estado_civil_agente","numero_de_hijos_agente","nivel_de_estudios_culminados_agente","celular_agente","correo_agente","direccion_agente","promedio_de_ingresos_agente"]

In [3]:
df1= pd.read_csv('../sintetic_ds.csv', sep=",")
df2= pd.read_csv('../sintetic_ds2.csv', sep=",")
df = pd.concat([df1,df2] , ignore_index=True)

#df= pd.read_csv('./sintetic_ds.csv', sep=";", header=None, names=header_list)
df.head()

,PH,TEMP,SOC,KEX,PBRAY,N_base_supply,P_base_supply,K_base_supply,N_recovery,P_recovery,K_recovery,NrTrees,crop,NminStore,NminVeg,NmaxStore,NmaxVeg,PminStore,PminVeg,PmaxStore,PmaxVeg,KminStore,KminVeg,KmaxStore,KmaxVeg,Unnamed: 25,N,P,K,No,Po,Ko,leaf_att,stem_att,store_att,SeasonLength,leaf_lim,stem_lim,store_lim,N_supply,P_supply,K_supply,N_uptake,P_uptake,K_uptake,N_gap,P_gap,K_gap
0,4.7,21,30,1.5,1.1,63.8,1.7,21.2,0.7,0.17,0.7,3810,Coffee,0.0028,0.0454,0.0048,0.0459,0.00038,0.0038,0.00028,0.003,0.0045,0.003,0.0064,0.004,6779.22444,230,130,130,0,0,0,3264.08034,3515.1441,3713.394876,240,3264.1,3515.1,3152.1,212.1,22.5,105.8,195.8,22.0,97.8,159.3,10.7,0.0
1,5.7,21,20,1.7,1.1,67.5,5.3,29.2,0.7,0.17,0.7,5210,Coffee,0.0028,0.0454,0.0048,0.0459,0.00038,0.0038,0.00028,0.003,0.0045,0.003,0.0064,0.004,9270.27804,200,50,310,0,0,0,4463.47994,4806.7981,4262.349226,240,4463.5,4806.8,2881.2,207.1,13.8,245.8,175.7,13.7,138.4,331.8,112.6,0.0
2,5.1,21,21,1.5,1.1,55.1,3.7,28.0,0.7,0.17,0.7,3090,Coffee,0.0028,0.0454,0.0048,0.0459,0.00038,0.0038,0.00028,0.003,0.0045,0.003,0.0064,0.004,5498.11116,550,80,310,0,0,0,2647.24626,2850.8649,3386.065776,240,2647.2,2850.9,3185.0,375.9,14.8,209.2,265.1,14.8,139.2,0.0,29.6,0.0
3,5.9,21,25,0.3,1.1,90.6,6.8,3.9,0.7,0.17,0.7,4050,Coffee,0.0028,0.0454,0.0048,0.0459,0.00038,0.0038,0.00028,0.003,0.0045,0.003,0.0064,0.004,7206.26220,460,130,80,0,0,0,3469.69170,3736.5705,3815.710656,240,3469.7,3736.6,3285.6,397.7,27.9,57.7,292.9,25.2,57.0,0.0,0.0,0.0
4,5.3,21,10,1.1,1.1,28.8,2.4,41.4,0.7,0.17,0.7,6300,Coffee,0.0028,0.0454,0.0048,0.0459,0.00038,0.0038,0.00028,0.003,0.0045,0.003,0.0064,0.004,11209.74120,280,40,50,0,0,0,5397.29820,5812.4430,4609.717375,240,5397.3,5812.4,1784.7,209.6,8.6,71.2,150.7,8.6,58.9,456.6,182.7,0.0


In [4]:
TO_KEEP=['PH', 'SOC', 'KEX', 'PBRAY', 'NrTrees', 'N', 'P', 'K', 'store_att', 'N_supply']
df = df[TO_KEEP]
df


,PH,SOC,KEX,PBRAY,NrTrees,N,P,K,store_att,N_supply
0,4.7,30,1.5,1.1,3810,230,130,130,3713.394876,212.1
1,5.7,20,1.7,1.1,5210,200,50,310,4262.349226,207.1
2,5.1,21,1.5,1.1,3090,550,80,310,3386.065776,375.9
3,5.9,25,0.3,1.1,4050,460,130,80,3815.710656,397.7
4,5.3,10,1.1,1.1,6300,280,40,50,4609.717375,209.6
...,...,...,...,...,...,...,...,...,...,...
14995,4.7,22,0.9,1.1,3030,680,150,320,3357.408336,441.6
14996,5.3,26,0.7,1.1,5980,520,150,190,4515.003455,421.9
14997,5.6,34,1.1,1.1,4710,530,120,230,4079.563476,479.9
14998,4.7,15,1.1,1.1,5290,630,10,60,4290.226726,471.3


In [5]:
df.count()

PH           15000
SOC          15000
KEX          15000
PBRAY        15000
NrTrees      15000
N            15000
P            15000
K            15000
store_att    15000
N_supply     15000
dtype: int64

## Escogiendo columnas relevantes

In [6]:
df.value_counts()
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   PH         15000 non-null  float64
 1   SOC        15000 non-null  int64  
 2   KEX        15000 non-null  float64
 3   PBRAY      15000 non-null  float64
 4   NrTrees    15000 non-null  int64  
 5   N          15000 non-null  int64  
 6   P          15000 non-null  int64  
 7   K          15000 non-null  int64  
 8   store_att  15000 non-null  float64
 9   N_supply   15000 non-null  float64
dtypes: float64(5), int64(5)
memory usage: 1.1 MB


In [14]:
y = df['N_supply']
#x= df
X = df.drop('N_supply', axis=1)
X.info()

X_train, X_test, y_train, y_test = train_test_split(X,y , test_size = 0.3, random_state = 0)

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   PH         15000 non-null  float64
 1   SOC        15000 non-null  int64  
 2   KEX        15000 non-null  float64
 3   PBRAY      15000 non-null  float64
 4   NrTrees    15000 non-null  int64  
 5   N          15000 non-null  int64  
 6   P          15000 non-null  int64  
 7   K          15000 non-null  int64  
 8   store_att  15000 non-null  float64
dtypes: float64(4), int64(5)
memory usage: 1.0 MB


In [16]:
print("\n")
print('\033[1m' + "N_supply Random Forest")
print('\033[0m')

#X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)

regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(X, y)
regr.get_params()

#
#print("Puntaje Random Forest Regressor: " + str(regr.oob_score_))
pred =  regr.predict(X_test)
gt = y_test


print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(gt, pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(gt, pred))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(gt, pred)))
mape = np.mean(np.abs((gt - pred) / np.abs(gt)))
print('Mean Absolute Percentage Error (MAPE):', round(mape * 100, 2))
print('Accuracy:', round(100*(1 - mape), 2))
print('R^2:', metrics.r2_score(gt, pred))





N_supply Random Forest

Mean Absolute Error (MAE): 39.6102451349599
Mean Squared Error (MSE): 2478.666084872407
Root Mean Squared Error (RMSE): 49.78620376040341
Mean Absolute Percentage Error (MAPE): 13.66
Accuracy: 86.34
R^2: 0.8040577448084347


In [9]:
print(regr.predict([[4.7, 30, 1.5, 1.1, 3810, 230, 130, 130, 3713.394876]]))

[194.62125707]


## Multi-layer Perceptron (MLP)

In [44]:

regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
pred = regr.predict(X_test)



print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(gt, pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(gt, pred))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(gt, pred)))
mape = np.mean(np.abs((gt - pred) / np.abs(gt)))
print('Mean Absolute Percentage Error (MAPE):', round(mape * 100, 2))
print('Accuracy:', round(100*(1 - mape), 2))
print('R^2:', metrics.r2_score(gt, pred))

regr.score(X_test, y_test)




Mean Absolute Error (MAE): 7.876929174468558
Mean Squared Error (MSE): 97.43605439415416
Root Mean Squared Error (RMSE): 9.870970286357576
Mean Absolute Percentage Error (MAPE): 2.65
Accuracy: 97.35
R^2: 0.9922975344071239


0.9922975344071239

In [20]:
print(regr.predict([[4.7, 30, 1.5, 1.1, 3810, 230, 130, 130, 3713.394876]]))

[227.10098147]


In [26]:
X_test.head()


,PH,SOC,KEX,PBRAY,NrTrees,N,P,K,store_att
1670,5.5,13,0.9,1.1,4950,410,120,180,4169.140656
13379,4.7,19,0.5,1.1,3070,190,110,180,3376.536886
10234,5.9,14,1.1,1.1,5710,210,50,270,4430.391226
4719,5.1,33,0.3,1.1,6360,290,40,240,4626.803920
7003,5.8,34,1.7,1.1,5400,400,150,240,4327.942000


In [27]:
y_test.head()

1670     327.6
13379    147.6
10234    193.8
4719     268.2
7003     396.4
Name: N_supply, dtype: float64

In [21]:
print(regr.predict([[5.1, 21, 1.5, 1.1, 3090, 550, 80, 310, 3386.065776]]))

[375.63776138]


In [28]:
print(regr.predict([[5.5, 13, 0.9, 1.1, 4950, 410, 120, 180, 4169.140656]]))

[320.64392561]


In [37]:
X_test[0:1]

,PH,SOC,KEX,PBRAY,NrTrees,N,P,K,store_att
1670,5.5,13,0.9,1.1,4950,410,120,180,4169.140656


In [39]:
display(X_test.iloc[0])

PH              5.500000
SOC            13.000000
KEX             0.900000
PBRAY           1.100000
NrTrees      4950.000000
N             410.000000
P             120.000000
K             180.000000
store_att    4169.140656
Name: 1670, dtype: float64